In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# NLTK Stop words
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

stop_words.extend(['from', 'subject', 're', 'edu', 'use', '-pron-',])
#'ningirsu', 'lugal', 'habura', 'lumma', 'dabrum', 'shulmushpa', 'inshushinak', 'lagash', 'ninsuna', 'eshnunna', 'zabshali', 'dumuzi', 'aanepada', 'simurrum', 'tirigan', 'enakale', 'urbillum', 'geshtinanna', 'ephur', 'shin', 'sippar', 'ephur', 'magur', 'shatillu', 'garta', 'alumidatum', 'nushushmar', 'alum', 'gisha', 'ninmarki', 'hendursag', 'guedena', 'hegal', 'erridupizir', 'ninshubur', 'ninhursaga', 'enlil', 'sargon', 'lugalmarda', 'shin', 'kazallu', 'push', 'bashar', 'shishil', 'dada', 'nisaba', 'dudu', 'eanatum', 'enmetena', 'ningal', 'dagan', 'urlumma', 'eridu', 'ibbisin', 'tiwa', 'nagarbi', 'abgal', 'zubi', 'meenlil', 'usarum', 'ilaba', 'annunitum', 'apiak', 'anshan', 'subartum', 'kigal', 'eresh', 'apiak', 'meenili', 'utu', 'magan', 'lugalkiginedudu', 'sharkalisharri', 'shin', 'gunidu', 'bagara', 'tirash', 'ningirsus', 'ebla', 'utu_hegal', 'utuhegal', 'ishkur', 'enanatum', 'baba', 'antasur', 'uruxkar', 'rec', 'gatumdu', 'nergal', 'simashki', 'lugal_uruxkar', 'enki', 'shara', 'uruk', 'nippur', 'mari', 'urukagina', 'numushda', 'inanna', 'naram', 'ishtar', 'kurgal', 'ninlil', 'whose', 'shusin', 'umma', 'nanshe', 'amarsin', 'rimush', 'naram_sin', 'girsu', 'shulgi', 'nammu', 'elam', 'agade', 'parahshum', 'shamash', 'susa', 'kish', 'simanum', 'kutha', 'gunida', 'gursar', 'nanna', 'adab', 'ningursaga', 'gutium', 'sheshgar', 'irnina', 'hibaritum', 'ishtar_annunitum', 'ninhursag', 'sin', 'abzu', 'epa', 'ibgal', 'edam', 'dilmun', 'kinir', 'emah', 'ningar', 'giguna', 'eamma', 'eras', 'manishtushu', 'urtur', 'kindazi', 'emeslam', 'emunusgisa', 'ninkununa', 'gipar', 'karzida', 'pasira', 'etemeniguru', 'geshtinana', 'armanum', 'euphrates', 'esar', 'amanus', 'enlil', 'puzur', 'eninsar', 'nissaba', 'rigmushdan', 'etunigaana', 'eduruenki', 'hiritum', 'lagardingirigi', 'kirilum', 'ekur', 'eninnu',' eada', 'eengur', 'imsag', 'zulum', 'igalim', 'eshdugru', 'ahush', 'ninsar', 'shin', 'lugalemush', 'urningirsu', 'enmezianna', 'kian', 'adad', 'eras'])
print(stop_words)

In [ ]:
#corpus

from glob import glob 
docs = []
for file_name in glob("input/*.txt"):
   with open(file_name, "rt") as ifd:
     for line in ifd:
       tokens = line.strip().split()
       docs.append(tokens)
print(docs)

In [ ]:
# Convert to list
data = list(docs)
print(data)

In [ ]:
#Tokenize and Clean-up

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
# step 12

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=500,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# step 13

# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis